# 🍏 기본 검색-증강 생성(RAG,Retrieval-Augmented Generation) 및 AIProjectClient 사용 🍎

이 노트북에서는 **기본 RAG** 플로우를 사용하여 데모를 수행합니다:
- **`azure-ai-projects`** (AIProjectClient)
- **`azure-ai-inference`** (임베딩, ChatCompletions)
- **`azure-ai-search`** (벡터 또는 하이브리드 검색용)

테마는 **건강 및 피트니스** 입니다. 🍏 간단한 건강 팁 세트를 만들어 임베드하고 검색 인덱스에 저장한 다음 관련 팁을 검색하는 쿼리를 수행하여 최종 답변을 생성하기 위해 LLM에 전달하겠습니다.

> **면책 조항**: 이 답변은 의학적 조언이 아닙니다. 실제 건강에 관한 질문은 전문가와 상담하세요.

## RAG란 무엇인가요?
Retrieval-Augmented Generation (RAG) 은 LLM(대규모 언어 모델)이 데이터에서 검색된 관련 텍스트 청크를 사용하여 최종 답변을 생성하는 기술입니다. 이는 모델의 응답을 실제 데이터에 근거하여 할루시네이션을 줄이는 데 도움이 됩니다.


<img src="./seq-diagrams/3-basic-rag.png" width="30%"/>

## 1. 설정

라이브러리를 가져오고, 환경 변수를 로드하고, `AIProjectClient`를 생성하겠습니다.


> #### 이 노트북을 시작하기 전에 [2-embeddings.ipynb](2-embeddings.ipynb) 노트북을 완성하세요.

In [ ]:
import os
import time
import json
from dotenv import load_dotenv

# azure-ai-projects
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# We'll embed with azure-ai-inference
from azure.ai.inference import EmbeddingsClient, ChatCompletionsClient
from azure.ai.inference.models import UserMessage, SystemMessage

# For vector search or hybrid search
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

conn_string = os.environ.get("PROJECT_CONNECTION_STRING")
chat_model = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
embedding_model = os.environ.get("EMBEDDING_MODEL_DEPLOYMENT_NAME", "text-embedding-3-small")
search_index_name = os.environ.get("SEARCH_INDEX_NAME", "healthtips-index")

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=conn_string,
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)

## 2. 샘플 건강 데이터 만들기
몇 개의 짧은 문서 청크를 만들어 보겠습니다. 실제 시나리오에서는 CSV 또는 PDF에서 읽고, 청크로 묶어 임베드하고, 검색 인덱스에 저장할 수 있습니다.


In [ ]:
health_tips = [
    {
        "id": "doc1",
        "content": "Daily 30-minute walks help maintain a healthy weight and reduce stress.",
        "source": "General Fitness"
    },
    {
        "id": "doc2",
        "content": "Stay hydrated by drinking 8-10 cups of water per day.",
        "source": "General Fitness"
    },
    {
        "id": "doc3",
        "content": "Consistent sleep patterns (7-9 hours) improve muscle recovery.",
        "source": "General Fitness"
    },
    {
        "id": "doc4",
        "content": "For cardio endurance, try interval training like HIIT.",
        "source": "Workout Advice"
    },
    {
        "id": "doc5",
        "content": "Warm up with dynamic stretches before running to reduce injury risk.",
        "source": "Workout Advice"
    },
    {
        "id": "doc6",
        "content": "Balanced diets typically include protein, whole grains, fruits, vegetables, and healthy fats.",
        "source": "Nutrition"
    },
]
print("Created a small list of health tips.")

## 3.0. 인덱스 만들기 또는 재설정
Azure AI Search에서 벡터 필드를 만들 때, **필드 정의**에 `vector_search_profile` 속성이 포함되어야 합니다. 이 속성은 벡터 검색 설정에서 일치하는 프로필 이름을 가리킵니다.

HNSW 알고리즘 구성으로 벡터 인덱스를 생성(또는 재설정)하는 도우미 함수를 정의하겠습니다.


In [3]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

def create_healthtips_index(
        endpoint: str, api_key: str, index_name: str, 
        dimension: int = 1536 # if using text-embedding-3-small
        ):
    """Create or update a search index for health tips with vector search capability."""
    
    index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(api_key))
    
    # Try to delete existing index
    try:
        index_client.delete_index(index_name)
        print(f"Deleted existing index: {index_name}")
    except Exception:
        pass  # Index doesn't exist yet
        
    # Define vector search configuration
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw"
            )
        ]
    )
    
    # Define fields
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SimpleField(name="source", type=SearchFieldDataType.String),
        SearchField(
            name="embedding", 
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=dimension,
            vector_search_profile_name="myHnswProfile" 
        ),
    ]
    
    # Create index definition
    index_def = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search
    )
    
    # Create the index
    index_client.create_index(index_def)
    print(f"✅ Created or reset index: {index_name}")

## 3.1. 색인 생성 및 건강 팁 업로드 🏋️

이제 팁을 업로드해 봅시다:
1. Azure AI Search에 **검색 연결 만들기**
2. 벡터 검색 기능으로 **인덱스 만들기**
3. 각 건강 팁에 대한 **임베딩 생성**
4. 임베딩과 함께 팁을 **업로드**합니다.

이렇게 하면 나중에 검색할 지식 기반이 만들어집니다. 인공지능 비서가 참조할 수 있는 '피트니스 라이브러리'를 구축한다고 생각하면 됩니다! 📚💪

In [ ]:
from azure.ai.projects.models import ConnectionType

# Step 1: Get search connection
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)
if not search_conn:
    raise RuntimeError("❌ No default Azure AI Search connection found!")
print("✅ Got search connection")

# Step 2: Get embeddings client and check embedding length
embeddings_client = project_client.inference.get_embeddings_client()
print("✅ Created embeddings client")

sample_doc = health_tips[0]
emb_response = embeddings_client.embed(
        model=embedding_model,
        input=[sample_doc["content"]]
    )
embedding_length = len(emb_response.data[0].embedding)
print(f"✅ Got embedding length: {embedding_length}")

# Step 3: Create the index
create_healthtips_index(
    endpoint=search_conn.endpoint_url,
    api_key=search_conn.key,
    index_name=search_index_name,
    dimension=embedding_length   # for text-embedding-3-large
)

# Step 4: Create search client for uploading documents
search_client = SearchClient(
    endpoint=search_conn.endpoint_url,
    index_name=search_index_name,
    credential=AzureKeyCredential(search_conn.key)
)
print("✅ Created search client")


# Step 5: Embed and upload documents
search_docs = []
for doc in health_tips:
    # Get embedding for document content
    emb_response = embeddings_client.embed(
        model=embedding_model,
        input=[doc["content"]]
    )
    emb_vec = emb_response.data[0].embedding
    
    # Create document with embedding
    search_docs.append({
        "id": doc["id"],
        "content": doc["content"],
        "source": doc["source"],
        "embedding": emb_vec,
    })

# Upload documents to index
result = search_client.upload_documents(documents=search_docs)
print(f"✅ Uploaded {len(search_docs)} documents to search index '{search_index_name}'")

## 4. 기본 RAG 흐름
### 4.1. 끌어내기
사용자가 쿼리하면:
1. 사용자 질문을 임베드합니다.
2. 해당 임베딩으로 벡터 인덱스를 검색하여 상위 문서를 가져옵니다.

### 4.2. 답변 생성
그런 다음 검색된 문서를 채팅 모델에 전달합니다.

> 실제 시나리오에서는 청크 및 요약에 대한 보다 고급 접근 방식을 사용할 수 있습니다. 여기에서는 간단한 방법을 다룹니다.


In [6]:
from azure.search.documents.models import VectorizedQuery

def rag_chat(query: str, top_k: int = 3) -> str:
    # 1) Embed user query
    user_vec = embeddings_client.embed(
        model=embedding_model,
        input=[query]).data[0].embedding

    # 2) Vector search using VectorizedQuery
    vector_query = VectorizedQuery(
        vector=user_vec,
        k_nearest_neighbors=top_k,
        fields="embedding"
    )

    results = search_client.search(
        search_text="",  # Optional text query
        vector_queries=[vector_query],
        select=["content", "source"]  # Only retrieve fields we need
    )

    # gather the top docs
    top_docs_content = []
    for r in results:
        c = r["content"]
        s = r["source"]
        top_docs_content.append(f"Source: {s} => {c}")

    # 3) Chat with retrieved docs
    system_text = (
        "You are a health & fitness assistant.\n"
        "Answer user questions using ONLY the text from these docs.\n"
        "Docs:\n"
        + "\n".join(top_docs_content)
        + "\nIf unsure, say 'I'm not sure'.\n"
    )

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=chat_model,
            messages=[
                SystemMessage(content=system_text),
                UserMessage(content=query)
            ]
        )
    return response.choices[0].message.content

## 5. 질문하기 🎉
바쁜 분들을 위해 유산소 운동에 관한 질문을 해보겠습니다.


In [ ]:
user_query = "What's a good short cardio routine for me if I'm busy?"
answer = rag_chat(user_query)
print("🗣️ User Query:", user_query)
print("🤖 RAG Answer:", answer)

## 6. 결론
지금까지 **기본 RAG** 파이프라인을 보여드렸습니다:
- 문서를 **임베드**하고 **Azure AI Search**에 저장하기.
- 사용자 질문에 대한 상위 문서 **끌어내기**
- 끌어낸 문서와 **채팅**

🔎 고급 청킹, 더 강력한 검색 및 품질 검사를 추가하여 확장할 수 있습니다. 🍎


🚀 작은 언어 모델로 더 최적화하고 싶으신가요? 다음 노트북 [4-phi-4.ipynb](4-phi-4.ipynb) 를 확인하여 동일한 Azure AI Foundry SDK를 사용하여 Phi-4를 사용하는 방법을 알아보세요!